In [1]:
import tensorflow as tf
import numpy as np
import os
from PIL import Image as pImage

pathDir = "./image"
fileList = os.listdir(pathDir)
print(fileList)

['yonsei_i.png', 'youtube_i.png']


In [2]:
X = tf.placeholder(tf.float32, [None, 300, 300, 1])
Y = tf.placeholder(tf.float32, [1])
isTraining = tf.placeholder(tf.bool)
global_step = tf.Variable(0, trainable=False, name='global_step')

In [3]:
L1 = tf.layers.conv2d(X, 32, [3, 3], activation=tf.nn.relu, padding='SAME')
L1 = tf.layers.max_pooling2d(L1, [2, 2], [2, 2])
L1 = tf.layers.dropout(L1, 0.7, isTraining)

In [4]:
L2 = tf.layers.conv2d(L1, 64, [3, 3], activation=tf.nn.relu, padding='SAME')
L2 = tf.layers.max_pooling2d(L2, [2, 2], [2, 2])
L2 = tf.layers.dropout(L2, 0.7, isTraining)

In [5]:
L3 = tf.contrib.layers.flatten(L2)
L3 = tf.layers.dense(L3, 256, activation=tf.nn.relu)
L3 = tf.layers.dropout(L3, 0.5, isTraining)

In [10]:
model = tf.layers.dense(L3, 1, activation=None)

In [11]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [15]:
init = tf.global_variables_initializer()
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())
ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(init)
for epoch in range(10):
    totalCost = 0
        
    for imgName in fileList:
        img = pImage.open(pathDir+"/"+imgName)
        img = img.resize((300, 300))
        img = img.convert("L")
        print(img)
        img.show()
        data = np.array(img).reshape(1, 300, 300, 1)
        print(data)
        target = []
        if imgName.split('_')[1] == "i":
            target = [1]
        else:
            target = [0]
        
        _, costVal = sess.run([optimizer, cost], feed_dict={X:data, Y:target, isTraining: True})
        totalCost += costVal
        
    print('Epoch: ', '%04d' % sess.run(global_step),
          'Avg. Cost: %3f' % (totalCost / 2))

<PIL.Image.Image image mode=L size=300x300 at 0x225CE9467B8>
[[[[255]
   [255]
   [255]
   ...
   [255]
   [255]
   [255]]

  [[255]
   [255]
   [255]
   ...
   [255]
   [255]
   [255]]

  [[255]
   [255]
   [255]
   ...
   [255]
   [255]
   [255]]

  ...

  [[255]
   [255]
   [255]
   ...
   [255]
   [255]
   [255]]

  [[255]
   [255]
   [255]
   ...
   [255]
   [255]
   [255]]

  [[255]
   [255]
   [255]
   ...
   [255]
   [255]
   [255]]]]
<PIL.Image.Image image mode=L size=300x300 at 0x225CE946C50>
[[[[255]
   [255]
   [255]
   ...
   [255]
   [255]
   [255]]

  [[255]
   [255]
   [255]
   ...
   [255]
   [255]
   [255]]

  [[255]
   [255]
   [255]
   ...
   [255]
   [255]
   [255]]

  ...

  [[255]
   [255]
   [255]
   ...
   [255]
   [255]
   [255]]

  [[255]
   [255]
   [255]
   ...
   [255]
   [255]
   [255]]

  [[255]
   [255]
   [255]
   ...
   [255]
   [255]
   [255]]]]
Epoch:  0000 Avg. Cost: 0.000000
<PIL.Image.Image image mode=L size=300x300 at 0x225CE946748>
[[[[255]
   

Epoch:  0000 Avg. Cost: 0.000000
<PIL.Image.Image image mode=L size=300x300 at 0x225CE9469B0>
[[[[255]
   [255]
   [255]
   ...
   [255]
   [255]
   [255]]

  [[255]
   [255]
   [255]
   ...
   [255]
   [255]
   [255]]

  [[255]
   [255]
   [255]
   ...
   [255]
   [255]
   [255]]

  ...

  [[255]
   [255]
   [255]
   ...
   [255]
   [255]
   [255]]

  [[255]
   [255]
   [255]
   ...
   [255]
   [255]
   [255]]

  [[255]
   [255]
   [255]
   ...
   [255]
   [255]
   [255]]]]
<PIL.Image.Image image mode=L size=300x300 at 0x225CE946940>
[[[[255]
   [255]
   [255]
   ...
   [255]
   [255]
   [255]]

  [[255]
   [255]
   [255]
   ...
   [255]
   [255]
   [255]]

  [[255]
   [255]
   [255]
   ...
   [255]
   [255]
   [255]]

  ...

  [[255]
   [255]
   [255]
   ...
   [255]
   [255]
   [255]]

  [[255]
   [255]
   [255]
   ...
   [255]
   [255]
   [255]]

  [[255]
   [255]
   [255]
   ...
   [255]
   [255]
   [255]]]]
Epoch:  0000 Avg. Cost: 0.000000


In [ ]:
saver.save(sess, './model/fald.ckpt', global_step=global_step)